# Get Lag Features

This code calculate lag features by using BigQuery API.

I'm creating three notebook similar to this one to avoid OOM.

These datasete is here:
- f0~99: https://www.kaggle.com/takamichitoda/ump-lag-1-features-f0f99
- f100~199: https://www.kaggle.com/takamichitoda/ump-lag-1-features-f100f199
- f200~299: https://www.kaggle.com/takamichitoda/ump-lag-1-features-f200f299

In [ ]:
from kaggle_secrets import UserSecretsClient
gcp_project_id = UserSecretsClient().get_secret("gcp_project_id")

In [ ]:
import gc
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from google.cloud import bigquery

In [ ]:
client = bigquery.Client(project=gcp_project_id)

In [ ]:
n_lag = 1

vecs = []
for feat_i in tqdm(range(0, 100)):
    query = f"""
    SELECT
        IFNULL(LAG (f_{feat_i}, {n_lag}) OVER (PARTITION BY investment_id ORDER BY time_id), 0) AS f_{feat_i}_l_{n_lag}
    FROM
        `{gcp_project_id}.ump.train`
    ORDER BY time_id, investment_id
    """

    query_job = client.query(query)

    data = query_job.to_dataframe()
    vecs.append(data.values[:, 0])

In [ ]:
np.save("f0_f99_lag_1", np.stack(vecs).T)

In [ ]:
!ls